In [189]:
from datetime import datetime, timedelta

In [190]:
class Material:
    def __init__(self, titulo, categoria):
        self.titulo= titulo
        self.categoria= categoria
        self.estado="disponible"
        self.fecha_prestamo = None
    
    def cambiar_estado(self, estado):
        self.estado= estado
        print(f'Estado de {self.titulo}: {self.estado}')    

    def detalles(self):
        print(f'Titulo: {self.titulo} |Estado: {self.estado}')

In [191]:
class Libro(Material):
    def __init__(self, titulo, autor, genero):
        super().__init__(titulo, "Libro")
        self.autor= autor
        self.genero= genero

    def detalles(self):
        print(f'Título: {self.titulo} |Autor: {self.autor} |Género: {self.genero} |Estado: {self.estado}')

In [192]:
class Revista(Material):
    def __init__(self, titulo, edicion, periocidad):
        super().__init__(titulo, "Revista")
        self.edicion= edicion
        self.periocidad= periocidad

    def detalles(self):
        print(f'Título: {self.titulo} |Edición: {self.edicion} |Periocidad: {self.periocidad} |Estado: {self.estado}')

In [193]:
class MaterialDigital(Material):
    def __init__(self, titulo, tipo_archivo, enlace):
        super().__init__(titulo, "Material Digital")
        self.tipo_archivo= tipo_archivo
        self.enlace= enlace

    def detalles(self):
        print(f'Título: {self.titulo} |Tipo de archivo: {self.tipo_archivo} |Enlace: {self.enlace} ')

In [194]:
class Persona:
    listapersonas= []

    def __init__(self, nombre, correo):
        self.nombre= nombre
        self.correo= correo
        
    def registrar(self):
        Persona.listapersonas.append(self)
        print(f'Se registró correctamente a {self.nombre} con correo de contacto {self.correo}')
    
    def cambiarnombre(self, nombrenuevo):
        self.nombre=nombrenuevo
        print(f'Se ha actualizado el nombre a: {self.nombre}')
    def cambiarcorreo(self, correonuevo):
        self.correo= correonuevo
        print(f'Se ha actualizado el correo a: {self.correo}')
 
    @classmethod
    def personas_registradas(cls):
        print("Personas registradas:")
        for persona in cls.listapersonas:
            print(f"-{persona.nombre} - {persona.correo}")

In [195]:
class Usuario(Persona):
    def __init__(self, nombre, correo):
        super().__init__(nombre, correo)
        self.materiales_prestados= []
    
    def consultar_catálogo(self, catalogo):
        catalogo.mostrar_materiales()
        
    def recibir_material(self, material):
        if material.estado == "disponible":
            self.materiales_prestados.append(material)
            material.cambiar_estado("prestado")
            print(f"{self.nombre} ha recibido el material: {material.titulo}")
        else:
            print(f"El material '{material.titulo}' no está disponible.")

    def devolver_material(self, material, fecha_devolucion= None):
        if material in self.materiales_prestados:
            self.materiales_prestados.remove(material)
            material.cambiar_estado("disponible")
            print(f"{self.nombre} ha devuelto el material: {material.titulo}")

            if fecha_devolucion is None:
                fecha_devolucion= datetime.now()
            else: fecha_devolucion= datetime.strptime(fecha_devolucion, "%d/%m/%Y")

            fecha_prestamo= material.fecha_prestamo
            fecha_limite= fecha_prestamo + timedelta(days=7)

            if fecha_devolucion > fecha_limite:
                retraso= (fecha_devolucion- fecha_limite).days
                print(f'{self.nombre} ha devuelto el material {material.titulo} con {retraso} días de retraso')
                penalizacion= Penalizacion(monto=3)
                penalizacion.agregar_penalizacion(self, retraso)
            else:
                print("Ha devuelto el material a tiempo")
        else:
            print(f"{self.nombre} no ha pedido el material '{material.titulo}' prestado.")
    

In [196]:
class Bibliotecario(Persona):
    def __init__(self, nombre, correo):
        super().__init__(nombre, correo)

    def agregar_material(self, material, sucursal):
        sucursal.agregar_material(material)

    def gestionar_prestamo(self, usuario, material):
        if material.estado == "disponible":
            usuario.recibir_material(material)
        else:
            print(f"El material '{material.titulo}' no está disponible.")


In [197]:
class Catalogo:
    def __init__(self, sucursal):
        self.sucursal = sucursal
        self.materiales = []

    def agregar_material(self, material):
        self.materiales.append(material)

    def mostrar_materiales(self):
        print(f"Catálogo de {self.sucursal.nombre}:")
        for material in self.materiales:
           print(f' {material.detalles()}')

    def buscar_material(self, criterio):
        print(f"Resultados de búsqueda para '{criterio}':")
        found= False
        for material in self.materiales:
            if criterio.lower() in material.titulo.lower():
                print(f"- {material.detalles()}")
                found= True

        if not found:
            print("No se encontraron materiales que coincidan con su búsqueda")

In [198]:
class Sucursal:
    def __init__(self, nombre):
        self.nombre= nombre
        self.catalogo= Catalogo(self)

    def agregar_material(self, material):
        self.catalogo.agregar_material(material)

In [199]:
class Prestamo:
    def __init__(self, usuario, material, fecha_prestamo= None, dias_devolucion=7):
        self.usuario = usuario
        self.material = material
         
        if fecha_prestamo:
            self.fecha= datetime.strptime(fecha_prestamo, "%d/%m/%Y")
        else:
            self.fecha= datetime.now()
        
        self.material.fecha_prestamo= self.fecha
        
        self.fecha_devolucion= self.fecha + timedelta(days= dias_devolucion)

    def devolver(self):
        self.material.cambiar_estado("disponible")
        print(f"El material '{self.material.titulo}' ha sido devuelto por {self.usuario.nombre}")
    


In [200]:
class Penalizacion:
    def __init__(self, monto=3):
        self.penalizaciones = {}
        self.monto= monto

    def agregar_penalizacion(self, usuario, dias_retraso):
        if usuario not in self.penalizaciones:
            self.penalizaciones[usuario] = 0
        self.penalizaciones[usuario] += dias_retraso * self.monto
        print(f"{usuario.nombre} tiene una penalización de ${self.penalizaciones[usuario]}.")

    def mostrar_penalizaciones(self):
        for usuario, monto in self.penalizaciones.items():
            print(f"{usuario.nombre}: ${monto}")

In [204]:
Usuario1= Usuario("Fernanda", "fer123@buap.mx")
Usuario1.registrar()
Bibliotecario1= Bibliotecario("Carlos", "carlos321@buap.mx")
Bibliotecario1.registrar()
Libro1= Libro("El lorax", "Dr. Seus", "Infantil")
Revista1 = Revista("Chismes", "II", "Mensual")
MaterialDigital1= MaterialDigital("Libro de Álgebra Lineal", "PDF", "Libro-de-Álgebra-Lineal.pdf")
Sucursal1= Sucursal("Sucursal Benito")
Sucursal2= Sucursal("Sucursal Juarez")
Bibliotecario1.agregar_material(Libro1, Sucursal1)
Catalogo1=Catalogo(Sucursal1)
Catalogo2= Catalogo(Sucursal2)
Catalogo1.agregar_material(Revista1)
Catalogo1.agregar_material(MaterialDigital1)
Usuario1.consultar_catálogo(Catalogo1)
Prestamo1= Prestamo(Usuario1, Libro1, "10/2/2025")
Usuario1.recibir_material(Libro1)
Usuario1.devolver_material(Libro1, "20/2/2025")
Catalogo1.buscar_material("chismes")
Catalogo1.buscar_material("álgebra")
Catalogo1.buscar_material("Lorax")

Se registró correctamente a Fernanda con correo de contacto fer123@buap.mx
Se registró correctamente a Carlos con correo de contacto carlos321@buap.mx
Catálogo de Sucursal Benito:
Título: Chismes |Edición: II |Periocidad: Mensual |Estado: disponible
 None
Título: Libro de Álgebra Lineal |Tipo de archivo: PDF |Enlace: Libro-de-Álgebra-Lineal.pdf 
 None
Estado de El lorax: prestado
Fernanda ha recibido el material: El lorax
Estado de El lorax: disponible
Fernanda ha devuelto el material: El lorax
Fernanda ha devuelto el material El lorax con 3 días de retraso
Fernanda tiene una penalización de $9.
Resultados de búsqueda para 'chismes':
Título: Chismes |Edición: II |Periocidad: Mensual |Estado: disponible
- None
Resultados de búsqueda para 'álgebra':
Título: Libro de Álgebra Lineal |Tipo de archivo: PDF |Enlace: Libro-de-Álgebra-Lineal.pdf 
- None
Resultados de búsqueda para 'Lorax':
No se encontraron materiales que coincidan con su búsqueda
